In [1]:
%%time
import pyarrow.parquet as pq
import pandas as pd
import re
from tqdm import tqdm 
import torch
from transformers import AutoTokenizer, BertTokenizer, pipeline
tqdm.pandas(desc="Processing excerpts")



CPU times: total: 3.86 s
Wall time: 21.6 s


In [2]:
# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')
classifier('We are very happy to include pipeline into the transformers repository.')
[{'label': 'POSITIVE', 'score': 0.9978193640708923}]


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9978193640708923}]

### Load dataset

In [3]:
filepath = './Dataset/bloomberg_news.csv'
df = pd.read_csv(filepath)
df = df.drop(df.columns[0], axis=1)
df.rename(columns={'0':'times', '1':'articles'}, inplace=True)
print(df)

             times                                           articles
0       2006-10-20  -- Inco's Net Soars on Higher Metal Prices, Br...
1       2006-10-21  -- Jim Cramer: Diageo, Anheuser-Busch, Monster...
2       2006-10-23  -- EU Energy Chief Backs German Plan for Price...
3       2006-10-23  -- Ex-Plant Worker Shuster Pleads Guilty in Tr...
4       2006-10-24  -- Jim Cramer: Bare Escentuals, Allergan, Medi...
...            ...                                                ...
448390  2013-11-26  -- Rubber Drops to Two-Week Low After Forecast...
448391  2013-11-26  -- SNB’s Jordan Sees No Reason to Remove Cap o...
448392  2013-11-26  -- U.K. Lawmakers Seek ‘Sharp’ Change as Bank ...
448393  2013-11-26  -- UBS Offers to Repurchase Some Shares of Pue...
448394  2013-11-26  -- Hyundai Motor Unveils New Genesis to Boost ...

[448395 rows x 2 columns]


In [4]:
txt_length = df['articles'].apply(lambda x: len(x) if pd.notnull(x) else 0)
df['articles_len'] = txt_length

In [5]:
df.head()

,times,articles,articles_len
0,2006-10-20,"-- Inco's Net Soars on Higher Metal Prices, Br...",4459
1,2006-10-21,"-- Jim Cramer: Diageo, Anheuser-Busch, Monster...",3026
2,2006-10-23,-- EU Energy Chief Backs German Plan for Price...,3691
3,2006-10-23,-- Ex-Plant Worker Shuster Pleads Guilty in Tr...,2977
4,2006-10-24,"-- Jim Cramer: Bare Escentuals, Allergan, Medi...",3724


In [6]:
df['articles'].fillna('', inplace=True) 

In [7]:
df = df[df['times']>= '2010-10-01']
df.describe()

,articles_len
count,418572.000000
mean,3039.007272
std,2702.060162
min,0.000000
25%,1233.000000
50%,2277.000000
75%,4069.000000
max,96656.000000


In [8]:
df.head()

,times,articles,articles_len
29823,2010-10-01,-- `Cowboy' Code Name Hid Southwest in Talks t...,7155
29824,2010-10-01,-- ‘Freakonomics’ Film Features Cheating Sumo ...,4821
29825,2010-10-01,-- ABC-Mart May Report 10% Rise in Six-Month O...,627
29826,2010-10-01,-- Abramovich’s Company Asks Ireland to Honor ...,2444
29827,2010-10-01,-- AEP Says Deadlines for Cutting Power Plant ...,2107


### Calculate sentiment score

In [9]:
# Initialize DistilBERT tokenizer and sentiment analysis model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
sentiment_analysis = pipeline("sentiment-analysis", model="distilbert-base-uncased", tokenizer=tokenizer)

def Calculate_sentiment_score(text, max_length):
    # Split the text
    words = text.split()
    subtexts = ['']
    for word in words:
        # Check if it exceeds the maximum length and split the text
        if len(tokenizer.encode(subtexts[-1] + word)) < max_length:
            subtexts[-1] += word + ' '
        else:
            subtexts.append(word + ' ')

    # Create a DataFrame to store the results
    Sentiment_df = pd.DataFrame(columns=['subtext', 'label', 'score', 'length', 'weighted_score'])

    # Perform sentiment analysis on each text segment
    for subtext in subtexts:
        if subtext.strip():  # Ignore empty text segments
            result = sentiment_analysis(subtext)[0]
            length = len(subtext.split())  # Calculate the length of the text segment (number of words)
            weighted_score = result['score'] * length  # Calculate the weighted score
            new_data = {'subtext': subtext, 'label': result['label'], 'score': result['score'], 'length': length,
                        'weighted_score': weighted_score}
            new_df = pd.DataFrame([new_data])  # Create a DataFrame containing the new data
            # Use the concat method to add the new DataFrame to the original DataFrame
            df_list = [Sentiment_df, new_df]
            Sentiment_df = pd.concat([df for df in df_list if not df.empty])

    return Sentiment_df

def AVG_Sentiment_score(df):
    total_length = df['length'].sum()
    total_score = 0
    
    for index, row in df.iterrows():
        if row['label'] == 'NEGATIVE':
            total_score -= row['weighted_score']
        elif row['label'] == 'POSITIVE':
            total_score += row['weighted_score']
        
    weighted_average_score = total_score / total_length if total_length > 0 else 0
    
    return weighted_average_score

In [10]:
Sentiment_Scores = []
last_month = None

for times, group in df.groupby('times'):
    current_month = times[5:7]  # Get the current month
    score_perday = []
    if current_month != last_month:
        print("The current month is different from the previous month")
        # Save sentiment scores to a local CSV file by month
        file_name = f"./DistilBert/sentiment_scores_{pd.to_datetime(times).strftime('%Y-%m-%d')}.csv"
        Sentiment_Scores_df = pd.DataFrame(Sentiment_Scores, columns=['sentiment_scores','times'])
        Sentiment_Scores_df.to_csv(file_name)
        print("Saved")
        
        # Reset the Sentiment_Scores list to store scores for the next month
        Sentiment_Scores = []
        last_month = current_month  # Save the current month as the last month
        
    for article in group['articles']:        
        max_length = 500
        Sentiment_df = Calculate_sentiment_score(article, max_length)  # Calculate sentiment scores for each article
        print(Sentiment_df)
        weighted_average_score = AVG_Sentiment_score(Sentiment_df)  # Calculate the weighted average score for each article
        score_perday.append(weighted_average_score) 
        print(f"Time: {times}, Weighted Average Score: {weighted_average_score}")
    # Append the list of sentiment scores for a day
    Sentiment_Scores.append([score_perday,times])
    

当前月份与上一个月份不同
Saved
Time: 2010-10-01, Weighted Average Score: -0.28085012408450194
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: -0.8135281801223755
Time: 2010-10-01, Weighted Average Score: -0.5807483512958973
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.9063062071800232
Time: 2010-10-01, Weighted Average Score: 0.6841454945920363
Time: 2010-10-01, Weighted Average Score: -0.7365878820419312
Time: 2010-10-01, Weighted Average Score: -0.5010348137021772
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: -0.3830944375540654
Time: 2010-10-01, Weighted Average Score: -0.8264645126480118
Time: 2010-10-01, Weighted Average Score: -0.5273762699790988
Time: 2010-10-01, Weighted Average Score: -0.8008806073623496
Time: 2010-10-01, Weighted Average Score: -0.22860229889625114
Time: 2

Time: 2010-10-01, Weighted Average Score: -0.1515978636861849
Time: 2010-10-01, Weighted Average Score: -0.3313993049992455
Time: 2010-10-01, Weighted Average Score: -0.15787583767558655
Time: 2010-10-01, Weighted Average Score: -0.561537924422521
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: -0.6036361479489817
Time: 2010-10-01, Weighted Average Score: -0.7505389517051164
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: -0.8559991717338562
Time: 2010-10-01, Weighted Average Score: -0.21947602288518459
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.9024830030300687
Time: 2010-10-01, Weighted Average Score: -0.8115820885311562
Time: 2010-1

Time: 2010-10-01, Weighted Average Score: -0.8114334440369404
Time: 2010-10-01, Weighted Average Score: -0.9183965836013301
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.17836018261455355
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.2513119853882982
Time: 2010-10-01, Weighted Average Score: -0.9128879904747009
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: 0.0
Time: 2010-10-01, Weighted Average Score: -0.8500980863741032
Time: 2010-10-01, Weighted Average Score: -0.7567068934440613
Time: 2010-10-0

Time: 2010-10-02, Weighted Average Score: -0.8360632848275881
Time: 2010-10-02, Weighted Average Score: -0.527488355283384
Time: 2010-10-02, Weighted Average Score: -0.690474510731062
Time: 2010-10-02, Weighted Average Score: -0.44663412876175224
Time: 2010-10-02, Weighted Average Score: -0.27927145767845585
Time: 2010-10-02, Weighted Average Score: 0.0
Time: 2010-10-02, Weighted Average Score: -0.9012742671224501
Time: 2010-10-02, Weighted Average Score: 0.42434991375681314
Time: 2010-10-02, Weighted Average Score: 0.813161313533783
Time: 2010-10-02, Weighted Average Score: 0.0
Time: 2010-10-02, Weighted Average Score: 0.0
Time: 2010-10-02, Weighted Average Score: 0.5425927042961121
Time: 2010-10-02, Weighted Average Score: -0.33936758239501147
Time: 2010-10-02, Weighted Average Score: -0.8060622172508282
Time: 2010-10-02, Weighted Average Score: -0.9572985172271729
Time: 2010-10-02, Weighted Average Score: -0.7110453248023987
Time: 2010-10-02, Weighted Average Score: 0.0
Time: 2010-1

Time: 2010-10-03, Weighted Average Score: -0.7315147343803855
Time: 2010-10-03, Weighted Average Score: -0.8075729203103158
Time: 2010-10-03, Weighted Average Score: -0.4696482562897157
Time: 2010-10-03, Weighted Average Score: -0.09761422895781013
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: -0.7014474158795547
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: 0.6477017650850638
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: -0.9718235731124878
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: 0.7995393872261047
Time: 2010-10-03, Weighted Average Score: -0.2027590372629054
Time: 2010-10-03, Weighted Average Score: -0.31411570896583635
Time: 2010-10-03, Weighted Average Score: 0.0
Time: 2010-10-03, Weighted Average Score: -0.0990361773284

...

Time: 2010-11-01, Weighted Average Score: -0.538212239742279
Time: 2010-11-01, Weighted Average Score: 0.7240887813662228
Time: 2010-11-01, Weighted Average Score: -0.5977772826551224
Time: 2010-11-01, Weighted Average Score: -0.8648588299125511
Time: 2010-11-01, Weighted Average Score: -0.7179048313544347
Time: 2010-11-01, Weighted Average Score: -0.770144779727144
Time: 2010-11-01, Weighted Average Score: -0.3109082260302135
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.3134014813169356
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: -0.09228091841598726
Time: 2010-11-01, Weighted Aver

Time: 2010-11-01, Weighted Average Score: -0.872087996990923
Time: 2010-11-01, Weighted Average Score: 0.939400315284729
Time: 2010-11-01, Weighted Average Score: 0.5096602281987272
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.8557975896050997
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.0
Time: 2010-11-01, Weighted Average Score: 0.5937515404141986
Time: 2010-11-01, Weighted Average Score: -0.3501368866452568
Time: 2010-11-01, Weighted Average Score: -0.9629966616630554
Time: 2010-11-01, Weighted Average Score: 0.9233068227767944
Time: 2010-11-01, Weighted Average Score: 0.9173439741134644
Time: 2010-11-01, Weighted Average Score: 0.9529668688774109
Time: 2010-11-01, Weighted Average Score: 0.9353057742118835
Time: 2010-11-01, Weighted Average Score: 0.005747122829463206
Time: 2010-11-01, Weighted Average 

Time: 2010-11-02, Weighted Average Score: -0.30276850512348635
Time: 2010-11-02, Weighted Average Score: 0.7105962494343662
Time: 2010-11-02, Weighted Average Score: 0.9431357979774475
Time: 2010-11-02, Weighted Average Score: 0.9335185289382935
Time: 2010-11-02, Weighted Average Score: 0.5405357601691265
Time: 2010-11-02, Weighted Average Score: 0.9200555086135864
Time: 2010-11-02, Weighted Average Score: 0.9265585063467433
Time: 2010-11-02, Weighted Average Score: 0.31956357222880233
Time: 2010-11-02, Weighted Average Score: -0.8119904398918152
Time: 2010-11-02, Weighted Average Score: 0.41808408748155856
Time: 2010-11-02, Weighted Average Score: -0.36364970069024816
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: -0.335026608861011
Time: 2010-11-02, Weighted Average Score: 0.593910457741501
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.7869200510356356
Time: 2010-11-02, Weighted Average Score: 0.7460

Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: -0.08869265764951706
Time: 2010-11-02, Weighted Average Score: 0.9272670745849609
Time: 2010-11-02, Weighted Average Score: -0.9439955949783325
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: -0.9717634320259094
Time: 2010-11-02, Weighted Average Score: 0.6459364230659875
Time: 2010-11-02, Weighted Average Score: 0.5691226215973444
Time: 2010-11-02, Weighted Average Score: -0.6733933466261831
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.0
Time: 2010-11-02, Weighted Average Score: 0.7850141175472922
Time: 2010-11-02, Weighted Average Score: 0.0



KeyboardInterrupt

